In [1]:
# from google.colab.patches import cv2_imshow
import cv2
import copy
import numpy as np
import os
import matplotlib.pyplot as plt
import mediapipe as mp
from google.protobuf.json_format import MessageToDict

In [2]:
import tensorflow as tf
from tensorflow import keras
from keras.utils import np_utils
import mediapipe as mp

In [3]:
SavedModel = keras.models.load_model('path_to_model') 

Capturing hands in real time for predicting


In [4]:
mp_drawing = mp.solutions.drawing_utils
mp_hands = mp.solutions.hands

In [5]:
#Some of the functions we'll be using
def get_bbox_coordinates(handLadmark, image_shape):
    
    all_x, all_y = [], [] # store all x and y points in list
    for hnd in mp_hands.HandLandmark:
        all_x.append(int(handLadmark.landmark[hnd].x * image_shape[1])) # multiply x by image width
        all_y.append(int(handLadmark.landmark[hnd].y * image_shape[0])) # multiply y by image height

    return min(all_x), min(all_y), max(all_x), max(all_y) # return as (xmin, ymin, xmax, ymax)

def return_letter(array):
    if(array[0][0].any()==1):
        return('A')
    elif(array[0][1].any()==1):
        return('B')
    elif(array[0][2].any()==1):
        return('C')
    elif(array[0][3].any()==1):
        return('D')
    elif(array[0][4].any()==1):
        return('E')
    elif(array[0][5].any()==1):
        return('F')
    elif(array[0][6].any()==1):
        return('G')
    elif(array[0][7].any()==1):
        return('H')
    elif(array[0][8].any()==1):
        return('I')
    elif(array[0][9].any()==1):
        return('J')
    elif(array[0][10].any()==1):
        return('K')
    elif(array[0][11].any()==1):
        return('L')
    elif(array[0][12].any()==1):
        return('M')
    elif(array[0][13].any()==1):
        return('N')
    elif(array[0][14].any()==1):
        return('O')
    elif(array[0][15].any()==1):
        return('P')
    elif(array[0][16].any()==1):
        return('Q')
    elif(array[0][17].any()==1):
        return('R')
    elif(array[0][18].any()==1):
        return('S')
    elif(array[0][19].any()==1):
        return('T')
    elif(array[0][20].any()==1):
        return('U')
    elif(array[0][21].any()==1):
        return('V')
    elif(array[0][22].any()==1):
        return('W')
    elif(array[0][23].any()==1):
        return('x')
    elif(array[0][24].any()==1):
        return('Y')
    else:
        return('Z')


In [8]:
i = 1

cap = cv2.VideoCapture(0)
_, frame = cap.read()
h, w, c = frame.shape

with mp_hands.Hands(static_image_mode=False, max_num_hands=2, min_detection_confidence=0.5) as hands:
    while cap.isOpened():
        success, image = cap.read()
        if not success:
            print("Ignoring empty camera frame.")
            continue

        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        results = hands.process(image)
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
            
        if results.multi_hand_landmarks:
            brect = []
            for hand_landmarks in results.multi_hand_landmarks:
                brect.append(get_bbox_coordinates(hand_landmarks, (h, w)))
                    
            if len(brect)==1:
                cv2.rectangle(image, (brect[0][0]-35, brect[0][1]-35), (brect[0][2]+35, brect[0][3]+35), (0, 255, 0), 2)
                roi = image[brect[0][1]-35:brect[0][3]+35, brect[0][0]-35:brect[0][2]+35]
                roi = cv2.flip(cv2.resize(roi, (128,128)), 1)
                prediction = SavedModel.predict(roi.reshape(1, 128, 128, 3))
                pred_array = np.int8(prediction == prediction.max())
                cv2.putText(image, return_letter(pred_array), (125, 50), cv2.FONT_HERSHEY_COMPLEX, 0.9, (0,0,0), 2)
                # print(pred_array)

            if len(brect)>1:
                cv2.rectangle(image, (min(brect[0][0], brect[1][0])-35, (min(brect[0][1], brect[1][1]))-35), (max(brect[0][2], brect[1][2])+35, (max(brect[0][3], brect[1][3]))+35), (0, 255, 0), 2)
                roi = image[min(brect[0][1], brect[1][1])-35:max(brect[0][3], brect[1][3])+35, min(brect[0][0], brect[1][0])-35:max(brect[0][2], brect[1][2])+35]
                roi = cv2.flip(cv2.resize(roi, (128,128)), 1)
                prediction = SavedModel.predict(roi.reshape(1, 128, 128, 3))
                pred_array = np.int8(prediction == prediction.max())
                cv2.putText(image, return_letter(pred_array), (125, 50), cv2.FONT_HERSHEY_COMPLEX, 0.9, (0,0,0), 2)
                # print(pred_array)

            # cv2.imshow('Testing', cv2.flip(roi, 1))
            cv2.imshow('Testing', image)
            i += 1
            if cv2.waitKey(25) & 0xFF == ord(' '):
                break

cap.release()
cv2.destroyAllWindows()